In [ ]:
# @zhangjoy
# @主题因子检验:分5组
# @20180330

In [1]:
import pandas as pd
factor_data_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/lda/data/stock_topicfactor_all.csv")

In [2]:
factor_data_df['name']
stock_list_STPT = [i for i in factor_data_df['name'].tolist() if "ST" in i or "PT" in i]
stock_list_STPT

['ST保千里', '*ST油服']

In [3]:
# 股票分组
STPT = ['ST保千里', '*ST油服'] # 剔除 ST/PT 股票
def stock_group(data,time):
    group = [] # 股票（每月）分组
    data_df = data[['name',time]] # /code
    data_df = data_df.sort_values(by = time,axis = 0,ascending = False) # 按201604降序排序
    stock_list = data_df['name'].tolist() # /code
    group1 = [i for i in stock_list[:60] if not i in STPT]
    group2 = [i for i in stock_list[60:120] if not i in STPT]
    group3 = [i for i in stock_list[120:180] if not i in STPT]
    group4 = [i for i in stock_list[180:240] if not i in STPT]
    group5 = [i for i in stock_list[240:300] if not i in STPT]
    group.append(group1)
    group.append(group2)
    group.append(group3)
    group.append(group4)
    group.append(group5)
    stock.append(group)

In [5]:
stock = []
time_list = ['201604','201605','201606','201607','201608','201609','201610','201611','201612','201701','201702','201703','201704','201705','201706','201707','201708','201709']
for time in time_list:
    stock_group(factor_data_df,time)
print(len(stock)) # 18个月
print(len(stock[0])) # 分成5组
print(len(stock[0][0])) # 每组60只股票
print([len(stock[0][i]) for i in range(5)])

18
5
60
[60, 59, 59, 60, 60]


In [6]:
# 辅助函数：两列表对应元素相乘并求和
def list_sum(a, b): 
    s = sum([a[i]*b[i] for i in range(len(a))])
    return s

In [7]:
# 获取交易日日期数据
s_name = '华夏银行'
path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
s_df = pd.read_csv(path).sort_values(by='datetime',axis=0, ascending=True) # 按日期升序排序
tradetime = s_df['datetime'].tolist() # 交易日数据

In [8]:
# 同时筛选股票
# 每月交易日开始日和结束日，周末、节假日不（开盘）交易
tradetime_start = ['2016-04-01','2016-05-03','2016-06-01','2016-07-01','2016-08-01','2016-09-01','2016-10-10','2016-11-01','2016-12-01','2017-01-03','2017-02-03','2017-03-01','2017-04-05','2017-05-02','2017-06-01','2017-07-03','2017-08-01','2017-09-01']
tradetime_end = ['2016-04-29','2016-05-31','2016-06-30','2016-07-29','2016-08-31','2016-09-30','2016-10-31','2016-11-30','2016-12-30','2017-01-26','2017-02-28','2017-03-31','2017-04-28','2017-05-31','2017-06-30','2017-07-31','2017-08-31','2017-09-29']
# 计算每个月 月末净（资产）值 + 每个月每组股票的持股数(/stay_num 登权分配)
def asset_month(group, asset_list, sharenum_list, staystock_list): # group组号
    asset = 1
    asset_group = []
    sharenum_group = []
    staystock_group = []
    for month in range(18):
        share_num = [] # 持股数
        end_close = [] # 月末收盘价，计算月末净值，即新的资产值
        stay_stock = [] # 筛选的股票
        for s in range(len(stock[month][group])):
            s_name = stock[month][group][s]
            path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
            try: 
                s_df = pd.read_csv(path)  # 读取文件数据
                index_o = s_df.datetime.tolist().index(tradetime_start[month]) # 月初 2016-04-01 的索引
                index_c = s_df.datetime.tolist().index(tradetime_end[month]) # 月末 2016-04-29 的索引

                s_close_start = s_df[index_o:index_o+1]['close'].values[0] # 月初收盘价, 收盘价买入
                s_num = asset/s_close_start # 根据收盘价计算个股的持股数
                
                s_close_end = s_df[index_c:index_c+1]['close'].values[0] # 月末收盘价，收盘价卖出
                
                stay_stock.append(s_name)
                share_num.append(s_num)
                end_close.append(s_close_end)
                
            except:
                pass
            
        asset = list_sum(share_num,end_close)/len(stay_stock)
        asset_group.append(asset)
        sharenum_group.append(share_num)
        staystock_group.append(stay_stock)
    asset_list.append(asset_group)
    sharenum_list.append(sharenum_group)
    staystock_list.append(staystock_group)

In [9]:
asset_list, sharenum_list, staystock_list = [], [], []
for i in range(5):
    asset_month(i, asset_list, sharenum_list, staystock_list)

In [10]:
# 绘制净值曲线：已知 asset_list, sharenum_list, staystock_list, tradetime
tradetime_list =[tradetime[0:20],tradetime[20:41],tradetime[41:61],tradetime[61:82],tradetime[82:105],tradetime[105:125],tradetime[125:141],tradetime[141:163],tradetime[163:185],tradetime[185:203],tradetime[203:221],tradetime[221:244],tradetime[244:262],tradetime[262:282],tradetime[282:304],tradetime[304:325],tradetime[325:348],tradetime[348:369]]
def netvalue(group, netvalue_list):
    netvalue_group = []
    for month in range(18):
        for tradeday in tradetime_list[month]: # month
            close = []
            for s_name in staystock_list[group][month]:
                path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
                try:
                    s_df = pd.read_csv(path)  # 读取文件数据
                    index_c = s_df.datetime.tolist().index(tradeday) # 交易日的索引
                    s_close = s_df[index_c:index_c+1]['close'].values[0] # 交易日收盘价
                except:
                    s_close = 0
                close.append(s_close)
            net_value = list_sum(sharenum_list[group][month],close)/len(sharenum_list[group][month])
            netvalue_group.append(net_value)
    netvalue_list.append(netvalue_group)

In [11]:
netvalue_list = []
for i in range(5):
    %time netvalue(i, netvalue_list)

CPU times: user 32.6 s, sys: 1.88 s, total: 34.5 s
Wall time: 34.7 s
CPU times: user 34.9 s, sys: 2.16 s, total: 37.1 s
Wall time: 37.7 s
CPU times: user 33 s, sys: 2.08 s, total: 35.1 s
Wall time: 35.5 s
CPU times: user 34.6 s, sys: 2.28 s, total: 36.8 s
Wall time: 37.3 s
CPU times: user 31.3 s, sys: 1.85 s, total: 33.2 s
Wall time: 33.4 s


In [12]:
# 计算策略指标
asset_start = 1
asset_end = [netvalue_list[i][-1] for i in range(5)] # 每组的最后资产值
return_list = [(i-asset_start)/asset_start for i in asset_end] # 总收益率
annual_return_list = [pow((1+i),250/369)-1 for i in return_list] # 年化收益率
# 总收益率
print(return_list)
# 年化收益率
print(annual_return_list) 

[0.0465467253686358, 0.12567874642208743, 0.01132543357453586, -0.0642079031733338, -0.07785696890480565]
[0.03130375977672939, 0.0835118098438965, 0.007659115034446495, -0.043964916123782394, -0.05343465142159387]


In [13]:
# 日收益率列表
dayreturn_list = []
for group in range(5):
    day_return = [(netvalue_list[group][i]-netvalue_list[group][i-1])/netvalue_list[group][i-1] for i in range(1,369)]
    firstday_return = netvalue_list[group][0]-1 # 将第1个交易日的日收益率插入日收益率列表中
    day_return.insert(0,firstday_return)
    dayreturn_list.append(day_return)

In [14]:
# 夏普比率
import numpy as np
sharpe_list = []
for i in range(5):
    dayreturn_array = np.array(dayreturn_list[i])
    sigma = np.std(dayreturn_array,ddof=1) # np.std无偏样本标准差方式为 ddof = 1 # 波动率计算
    sharpe = (return_list[i]-0.03)/sigma
    sharpe_list.append(sharpe)
sharpe_list

[1.3222548967337415,
 8.619479978304964,
 -1.4438584018513052,
 -9.291634075496521,
 -10.077478424866749]

In [16]:
# 胜率
def largerthan0_num(lst):
    num = 0
    for i in lst:
        if i >= 0:
            num = num + 1
    return num
print([largerthan0_num(dayreturn_list[i])/len(dayreturn_list[0]) for i in range(5)])

[0.5284552845528455, 0.5176151761517616, 0.5365853658536586, 0.47696476964769646, 0.5094850948509485]


In [17]:
# 最大回撤
# 第 num 天的最大回撤值
def drawdown(dayreturn_lst, num):
    return dayreturn_lst[num] - max(dayreturn_lst[0:num]) # 前 num 天
maxdrawdown_list =[]
for group in range(5):
    drawdown_list = [drawdown(dayreturn_list[group],i) for i in range(1,369)] # 368, 第1天没有回撤值
    maxdrawdown_list.append(max(drawdown_list))
maxdrawdown_list

[0.03560597584514482,
 0.024070519850608374,
 0.020011180503650694,
 0.018443868410791264,
 0.021280391864789028]